In [8]:
from __future__ import print_function
import librosa
import numpy as np
import cv2 as cv
import shutil
import glob
from sys import exit
import subprocess
from tkinter import filedialog
from tkinter import *
from os import listdir
from os.path import isfile, join
import math
import os
try:
    import requests
except ImportError:
    print('Requests must be installed. PLease run: pip install requests')
    exit()


y=''
sr=''
dir="reddit_images/"
fps = 30
vid_height=1080
vid_width = 1920



fourcc = cv.VideoWriter_fourcc(*'XVID')
out = cv.VideoWriter('output.avi', fourcc, 20.0, (vid_width,  vid_height))
def beats(song):
    filename = song

    # 2. Load the audio as a waveform `y`
    #    Store the sampling rate as `sr`
    y, sr = librosa.load(filename)

    # 3. Run the default beat tracker
    tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)

    print('Estimated tempo: {:.2f} beats per minute'.format(tempo))

    # 4. Convert the frame indices of beat events into timestamps
    beat_times = librosa.frames_to_time(beat_frames, sr=sr)
    return beat_times



    

def makeUrl(afterID, subreddit):
    newUrl = subreddit.split('/.json')[0] + "/.json?after={}".format(afterID)
    return newUrl
    

def splitUrl(imageUrl):
    if 'jpg' or 'webm' or 'mp4' or 'gif' or 'gifv' or 'png' in imageUrl:
        return imageUrl.split('/')[-1]  # Returns filename


def downloadImage(imageUrl, imageAmount):  # Function to download the images from
    global dir
    print(imageUrl)                          # each url passed to it
    filename = splitUrl(imageUrl)
    #print(filename)
    if filename:
        if ('jpg' or 'webm' or 'mp4' or 'gif' or 'gifv' or 'png') in filename:
            r = requests.get(imageUrl, stream=True)  # Downloads image
            with open(dir+filename, 'wb') as f:
                shutil.copyfileobj(r.raw, f)
                #print("Successfully downloaded: " + imageUrl)
                imageAmount += 1
    return imageAmount


def reddit_rip(subname,limit,sorting="hot",time="default"):
    subreddit = "https://www.reddit.com/r/"+subname+"/"+sorting
    if time != "default":
        subreddit+="/?t="+time
    print(subreddit)
    print(limit)
    limit=int(limit)
    #amount = int(input("How many images would you like to download: "))
    subJson = ''
    x = 0
    while x < limit:
        if subJson:
            url = makeUrl(subJson['data']['after'], subreddit)
        else:
            url = makeUrl('', subreddit)
        subJson = requests.get(url, headers={'User-Agent': 'MyRedditScraper'}).json()
        #print(subJson)
        post = subJson['data']['children']
        postCount = range(len(post))
        
        # Sets postCount equal to the number of posts

        for i in postCount:  # Iterates over the number of posts received the sub
            imageUrl = (post[i]['data']['url'])
            _imageUrls = []
            _imageUrls.append(imageUrl)
            x = downloadImage(_imageUrls[0], x)
            print(x)
            if x == limit:
                break
def makesize(img,targetsize):
    print(img.shape)
    print(targetsize)
    if targetsize[0]/img.shape[1] >=targetsize[1]/img.shape[0]:
        othersize=int(img.shape[1]*(targetsize[1]/img.shape[0]))
        
        img=cv.resize(img,(othersize,targetsize[1]))
        print(othersize)
    else:
        othersize=int(img.shape[0]*(targetsize[0]/img.shape[1]))
        
        img=cv.resize(img,(targetsize[0],othersize))
        print(othersize)
    print(img.shape)
    ht, wd, cc= img.shape
    color = (0,0,0)
    result = np.full((targetsize[1],targetsize[0],cc), color, dtype=np.uint8)
    print(result.shape)
    # compute center offset
    xx = (targetsize[0] - wd) // 2
    yy = (targetsize[1] - ht) // 2
    print(xx,yy)
    # copy img image into center of result image
    result[yy:yy+ht, xx:xx+wd] = img
    return result

def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles

def image_beat_match(beatss):
    global out
    global fps
    global flipflop
    imagelist = getListOfFiles(dir)
    print(imagelist)
    print(type(imagelist))
    for i in range(int(beatss[0]*fps)):
        frame=np.zeros((vid_height,vid_width,3), np.uint8)
        #cvSet(frame, CV_RGB(redVal,greenVal,blueVal));
        out.write(frame)
    for beat in beatss[1:]:
        print(beat)
        w=list(beatss).index(beat)
        print("w",w)
        if len(imagelist)<=w:
            print("not enough images. ending")
            break
        j=imagelist[w]
        img=cv.imread(j)
        frame=makesize(img,(vid_width,vid_height))
        print(int((beatss[(w-1)])*fps))
        for i in range(int((beatss[(w-1)])*fps),int(beat*fps)):  
            out.write(frame)
        
        
        
def run ():
    sub=input("What subreddit do you want to use? r/")
    global dir
    dir+=sub+"/"
    print(dir)
    if not os.path.exists(dir):
        os.makedirs(dir)
    #song input
    #root = Tk()
    #root.withdraw()
    #root.filename =  filedialog.askopenfilename(initialdir = ".",title = "Select file",filetypes = (("mp3 files","*.mp3"),("all files","*.*")))
    song="song.mp3"#root.filename #TODO fix this garbage
    #print (root.filename)
    #root.destroy()
    y, sr = librosa.load(song)
    reddit_rip(sub,len(beats(song)))
    print("image matching time")
    image_beat_match(beats(song))
    out.release
    os.system("cmd.exe /c ffmpeg -i output.avi -i "+song+" -map 0:0 -map 1:0 -c:v copy -c:a copy output_with_audio.avi")
    print("done")


run()

What subreddit do you want to use? r/hmmm
reddit_images/hmmm/
Estimated tempo: 172.27 beats per minute
https://www.reddit.com/r/hmmm/hot
736
https://i.redd.it/s8aunmm2xd051.png
0
https://i.redd.it/mtdbbeny0a051.jpg
1
https://i.redd.it/ggw1fpwndqy31.jpg
2
https://i.redd.it/hxn0intsyd051.jpg
3
https://i.redd.it/64n8vedx5e051.jpg
4
https://i.redd.it/c3cmjfpmre051.jpg
5
https://i.redd.it/zdjs0wwrie051.jpg
6
https://i.redd.it/ml92kmsaff051.jpg
7
https://i.redd.it/8zyyurwxke051.jpg
8
https://i.redd.it/nc0n3hgmae051.jpg
9
https://i.redd.it/jcet77jwl9051.jpg
10
https://i.redd.it/p3cvml6gie051.jpg
11
https://i.redd.it/jwldxr8xte051.jpg
12
https://i.redd.it/0k1o8idfmf051.jpg
13
https://i.redd.it/vo4y4zbig6051.jpg
14
https://i.redd.it/n6qgtkyuce051.jpg
15
https://i.redd.it/adwzbntbze051.jpg
16
https://i.redd.it/na9b8r1ucd051.png
16
https://i.redd.it/91f8c32l37051.jpg
17
https://i.redd.it/gvcfusu21f051.jpg
18
https://i.redd.it/fp28mak7ce051.jpg
19
https://i.redd.it/amqhl7vmtd051.jpg
20
https://i.r

174
https://i.redd.it/breagwtodam21.jpg
175
https://i.imgur.com/VXEafzk.jpg
176
https://i.redd.it/qpl61ji1r7s11.jpg
177
https://i.imgur.com/kmkTeVy.jpg
178
https://i.redd.it/r0de2iag7wz41.jpg
179
https://i.redd.it/dbwjtoea6uz41.jpg
180
https://i.redd.it/f7b6ighvgoz41.png
180
https://i.redd.it/0blkmnra9vz41.jpg
181
https://imgur.com/0U689Td.png
181
https://i.redd.it/p50u5952csz41.jpg
182
https://imgur.com/KTRCzMM.png
182
https://i.redd.it/ts88uouvbxz41.jpg
183
https://i.redd.it/nq2yaparjvz41.jpg
184
https://i.redd.it/uk2d27jtatz41.jpg
185
https://i.redd.it/eo10qjy16wz41.jpg
186
https://i.redd.it/wrwka7y3ouz41.jpg
187
https://i.redd.it/xqyuwgsjttz41.jpg
188
https://i.imgur.com/NJQBezz.jpg
189
https://i.redd.it/ot0nidjp9vz41.jpg
190
https://i.redd.it/p9lbqcsqbvz41.jpg
191
https://i.redd.it/yyjiqial5xz41.jpg
192
https://imgur.com/t1RXQ32.png
192
https://i.redd.it/z5wziogmyvz41.png
192
https://i.redd.it/yctn9zxuhuz41.png
192
https://i.redd.it/i8p7ymkdjwz41.png
192
https://imgur.com/57a8Rck.

343
https://i.redd.it/5m5q59xo7cz41.jpg
344
https://i.redd.it/nil9x7p1icz41.jpg
345
https://i.redd.it/aiehxvqjc9z41.jpg
346
https://imgur.com/5fG5BI6.png
346
https://i.redd.it/uzuqu7iq48z41.jpg
347
https://i.imgur.com/zQ7V344.jpg
348
https://i.redd.it/849r1lyb7cz41.jpg
349
https://i.redd.it/sh3po52kraz41.jpg
350
https://i.imgur.com/Ix639IK.png
350
https://i.redd.it/rs9i1kj0x6z41.png
350
https://i.imgur.com/lPQO7fi.jpg
351
https://i.redd.it/vftpul9hm8z41.jpg
352
https://i.redd.it/18qstaqwgaz41.jpg
353
https://imgur.com/fiu6XgS.png
353
https://i.redd.it/ekj0hzle1az41.jpg
354
https://i.redd.it/stwf4c3snaz41.jpg
355
https://imgur.com/FPdZvak.png
355
https://i.redd.it/090hk0tqyaz41.jpg
356
https://i.redd.it/oudx2m44r9z41.jpg
357
https://i.redd.it/dsvuztadg6z41.png
357
https://i.redd.it/93viwmz3obz41.jpg
358
https://i.redd.it/dfym1fcypbz41.jpg
359
https://i.redd.it/hdx93zwp99z41.jpg
360
https://i.redd.it/koamw3bnvaz41.jpg
361
https://i.redd.it/1fdlbaoxj9z41.jpg
362
https://imgur.com/dYygRRY.

495
https://i.redd.it/gdsncj7jkny41.jpg
496
https://i.redd.it/fwcjt4wuhmy41.jpg
497
https://i.redd.it/qlobr7u60jy41.jpg
498
https://imgur.com/tqAHvQp.png
498
https://i.redd.it/pvq9ga7w9iy41.png
498
https://i.imgur.com/W46YKBN.jpg
499
https://i.redd.it/rg84hs701my41.jpg
500
https://i.imgur.com/UeAbwQx.jpg
501
https://i.redd.it/hrmaj8c61iy41.jpg
502
https://i.redd.it/0jq49ndxvey41.jpg
503
https://i.redd.it/nkaugrl6nly41.jpg
504
https://i.redd.it/lxktjzlttly41.jpg
505
https://i.redd.it/rrlqwfqwkmy41.png
505
https://i.redd.it/vqp41975oly41.jpg
506
https://i.redd.it/i2ixzjg31my41.jpg
507
https://i.redd.it/rmwksa96yky41.jpg
508
https://i.redd.it/0r8fivd8xmy41.jpg
509
https://i.redd.it/1bbfc4towky41.png
509
https://i.redd.it/1r4lpnnv2my41.jpg
510
https://i.redd.it/qa2desa3wly41.jpg
511
https://i.redd.it/qwc3zhhikky41.jpg
512
https://i.redd.it/d4vvr9f6jly41.jpg
513
https://i.redd.it/bnq52tnzxly41.jpg
514
https://i.redd.it/66309qd20ny41.png
514
https://i.redd.it/fonq8h2qrly41.jpg
515
https://im

664
https://i.redd.it/nskhpk4fe7051.jpg
665
https://i.redd.it/1a68zdmmy2051.jpg
666
https://i.redd.it/yr96ewuhz3051.jpg
667
https://i.redd.it/al0az33j08051.jpg
668
https://i.redd.it/66gily4ij6051.png
668
https://i.redd.it/23ou891ok5051.jpg
669
https://i.redd.it/c21g4f1bv4051.jpg
670
https://i.redd.it/wrwoccrpk2051.jpg
671
https://imgur.com/RUn3Fg5.png
671
https://i.redd.it/jkev9yywl5051.jpg
672
https://i.redd.it/o8ij3y3co2051.jpg
673
https://i.redd.it/pglgzkdk40051.jpg
674
https://i.redd.it/rz2lxw8hd7051.png
674
https://i.redd.it/nkiusxi6j5051.jpg
675
https://imgur.com/0huezFM.png
675
https://i.redd.it/ufrxn0i6pd421.jpg
676
https://i.redd.it/uvs8zkcype721.jpg
677
https://i.redd.it/ixnndgj8bpz21.jpg
678
https://i.redd.it/k50mgb9pk2051.jpg
679
https://i.redd.it/hxol7zilt4051.jpg
680
https://i.redd.it/55j9w1cy93051.jpg
681
https://i.redd.it/bsqa9y0w54051.jpg
682
https://i.redd.it/20fjroqnv3051.jpg
683
https://i.redd.it/qclsj556a1051.jpg
684
https://i.redd.it/ecbwot6oa5051.jpg
685
https://

...

64.62113378684808
w 172
(469, 640, 3)
(1920, 1080)
1473
(1080, 1473, 3)
(1080, 1920, 3)
223 0
1928
64.94621315192744
w 173
(488, 600, 3)
(1920, 1080)
1327
(1080, 1327, 3)
(1080, 1920, 3)
296 0
1938
65.29451247165532
w 174
(1137, 1080, 3)
(1920, 1080)
1025
(1080, 1025, 3)
(1080, 1920, 3)
447 0
1948
65.64281179138322
w 175
(834, 994, 3)
(1920, 1080)
1287
(1080, 1287, 3)
(1080, 1920, 3)
316 0
1958
65.99111111111111
w 176
(360, 640, 3)
(1920, 1080)
1920
(1080, 1920, 3)
(1080, 1920, 3)
0 0
1969
66.31619047619047
w 177
(1170, 1080, 3)
(1920, 1080)
996
(1080, 996, 3)
(1080, 1920, 3)
462 0
1979
66.66448979591837
w 178
(960, 768, 3)
(1920, 1080)
864
(1080, 864, 3)
(1080, 1920, 3)
528 0
1989
67.01278911564626
w 179
(465, 703, 3)
(1920, 1080)
1632
(1080, 1632, 3)
(1080, 1920, 3)
144 0
1999
67.33786848072562
w 180
(960, 712, 3)
(1920, 1080)
801
(1080, 801, 3)
(1080, 1920, 3)
559 0
2010
67.68616780045352
w 181
(1152, 1920, 3)
(1920, 1080)
1800
(1080, 1800, 3)
(1080, 1920, 3)
60 0
2020
68.0112471655

93.39065759637188
w 257
(539, 640, 3)
(1920, 1080)
1282
(1080, 1282, 3)
(1080, 1920, 3)
319 0
2791
93.71573696145124
w 258
(744, 594, 3)
(1920, 1080)
862
(1080, 862, 3)
(1080, 1920, 3)
529 0
2801
94.06403628117914
w 259
(880, 960, 3)
(1920, 1080)
1178
(1080, 1178, 3)
(1080, 1920, 3)
371 0
2811
94.3891156462585
w 260
(869, 1125, 3)
(1920, 1080)
1398
(1080, 1398, 3)
(1080, 1920, 3)
261 0
2821
94.73741496598639
w 261
(859, 945, 3)
(1920, 1080)
1188
(1080, 1188, 3)
(1080, 1920, 3)
366 0
2831
95.06249433106576
w 262
(393, 392, 3)
(1920, 1080)
1077
(1080, 1077, 3)
(1080, 1920, 3)
421 0
2842
95.41079365079365
w 263
(731, 750, 3)
(1920, 1080)
1108
(1080, 1108, 3)
(1080, 1920, 3)
406 0
2851
95.73587301587301
w 264
(1111, 719, 3)
(1920, 1080)
698
(1080, 698, 3)
(1080, 1920, 3)
611 0
2862
96.08417233560091
w 265
(688, 720, 3)
(1920, 1080)
1130
(1080, 1130, 3)
(1080, 1920, 3)
395 0
2872
96.40925170068027
w 266
(786, 500, 3)
(1920, 1080)
687
(1080, 687, 3)
(1080, 1920, 3)
616 0
2882
96.734331065759

121.8118820861678
w 341
(1080, 1080, 3)
(1920, 1080)
1080
(1080, 1080, 3)
(1080, 1920, 3)
420 0
3643
122.13696145124716
w 342
(679, 680, 3)
(1920, 1080)
1081
(1080, 1081, 3)
(1080, 1920, 3)
419 0
3654
122.43882086167801
w 343
(685, 1203, 3)
(1920, 1080)
1896
(1080, 1896, 3)
(1080, 1920, 3)
12 0
3664
122.76390022675737
w 344
(1056, 1599, 3)
(1920, 1080)
1635
(1080, 1635, 3)
(1080, 1920, 3)
142 0
3673
123.08897959183673
w 345
(1704, 1242, 3)
(1920, 1080)
787
(1080, 787, 3)
(1080, 1920, 3)
566 0
3682
123.43727891156463
w 346
(663, 500, 3)
(1920, 1080)
814
(1080, 814, 3)
(1080, 1920, 3)
553 0
3692
123.78557823129252
w 347
(488, 650, 3)
(1920, 1080)
1438
(1080, 1438, 3)
(1080, 1920, 3)
241 0
3703
124.1338775510204
w 348
(1012, 828, 3)
(1920, 1080)
883
(1080, 883, 3)
(1080, 1920, 3)
518 0
3713
124.45895691609978
w 349
(422, 412, 3)
(1920, 1080)
1054
(1080, 1054, 3)
(1080, 1920, 3)
433 0
3724
124.80725623582767
w 350
(1113, 1125, 3)
(1920, 1080)
1091
(1080, 1091, 3)
(1080, 1920, 3)
414 0
3733

(495, 604, 3)
(1920, 1080)
1317
(1080, 1317, 3)
(1080, 1920, 3)
301 0
4468
149.62938775510204
w 424
(1209, 967, 3)
(1920, 1080)
863
(1080, 863, 3)
(1080, 1920, 3)
528 0
4479
149.97768707482993
w 425
(375, 500, 3)
(1920, 1080)
1440
(1080, 1440, 3)
(1080, 1920, 3)
240 0
4488
150.3027664399093
w 426
(647, 1024, 3)
(1920, 1080)
1709
(1080, 1709, 3)
(1080, 1920, 3)
105 0
4499
150.62784580498865
w 427
(297, 457, 3)
(1920, 1080)
1661
(1080, 1661, 3)
(1080, 1920, 3)
129 0
4509
150.97614512471657
w 428
(735, 750, 3)
(1920, 1080)
1102
(1080, 1102, 3)
(1080, 1920, 3)
409 0
4518
151.32444444444445
w 429
(475, 564, 3)
(1920, 1080)
1282
(1080, 1282, 3)
(1080, 1920, 3)
319 0
4529
151.6495238095238
w 430
(1064, 1080, 3)
(1920, 1080)
1096
(1080, 1096, 3)
(1080, 1920, 3)
412 0
4539
151.97460317460317
w 431
(410, 582, 3)
(1920, 1080)
1533
(1080, 1533, 3)
(1080, 1920, 3)
193 0
4549
152.32290249433106
w 432
(849, 700, 3)
(1920, 1080)
890
(1080, 890, 3)
(1080, 1920, 3)
515 0
4559
152.64798185941044
w 433
(3

177.1914739229025
w 506
(747, 750, 3)
(1920, 1080)
1084
(1080, 1084, 3)
(1080, 1920, 3)
418 0
5305
177.5397732426304
w 507
(1093, 1079, 3)
(1920, 1080)
1066
(1080, 1066, 3)
(1080, 1920, 3)
427 0
5315
177.86485260770976
w 508
(450, 600, 3)
(1920, 1080)
1440
(1080, 1440, 3)
(1080, 1920, 3)
240 0
5326
178.1899319727891
w 509
(622, 1056, 3)
(1920, 1080)
1833
(1080, 1833, 3)
(1080, 1920, 3)
43 0
5335
178.538231292517
w 510
(877, 1242, 3)
(1920, 1080)
1529
(1080, 1529, 3)
(1080, 1920, 3)
195 0
5345
178.86331065759637
w 511
(871, 1080, 3)
(1920, 1080)
1339
(1080, 1339, 3)
(1080, 1920, 3)
290 0
5356
179.21160997732426
w 512
(486, 750, 3)
(1920, 1080)
1666
(1080, 1666, 3)
(1080, 1920, 3)
127 0
5365
179.55990929705214
w 513
(1332, 1080, 3)
(1920, 1080)
875
(1080, 875, 3)
(1080, 1920, 3)
522 0
5376
179.90820861678006
w 514
(2048, 1925, 3)
(1920, 1080)
1015
(1080, 1015, 3)
(1080, 1920, 3)
452 0
5386
180.2332879818594
w 515
(664, 500, 3)
(1920, 1080)
813
(1080, 813, 3)
(1080, 1920, 3)
553 0
5397
18

In [ ]:
import cv2 as cv
import os
import numpy as np
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles
img=cv.imread(getListOfFiles(dir)[0])
cv.imshow("yeet",img)

In [5]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [10]:
print(beatss[0]*fps)

NameError: name 'beatss' is not defined